In [61]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


In [62]:
N_centr = 4
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="input/"
file_names=["my-m_ee_Run14AuAu_57th_new_19676_1072runs.root"] 
#m_ee_Run14AuAu_50th_new_19640_1072runs m_ee_Run14AuAu_57th_new_19676_1072runs
#m_ee_Run14AuAu_48th_new_19631_387runs m_ee_Run14AuAu_49th_new_19636_1072runs
#m_ee_Run14AuAu_46th_new_19619_999runs test_BG_v94_0 m_ee_Run14AuAu_47th_new_19623_1078runs 
sim_file_path="../sim/output/Newembed/sngl/"
primary_file_names=['phi_50M_v8.root']
hadron_file_names=['charged_pi_100M_v1.root']
dalitz_file_names=['dalitz_50M_v8.root']
conv_file_names=['photon_100M_v2.root']
ccbar_name = ['ccbar_50M_v2.root']
bbbar_name = ['bbbar_10M_v0.root']
sim_file_names = [dalitz_file_names[0], primary_file_names[0], conv_file_names[0], ccbar_name[0], bbbar_name[0]]
legend_names = ["real","dalitz","phi","#gamma #rightarrow ee","ccbar","bbbar"] 
hist_select_3D_names = ["DCA2_hist_0","DCA2_hist_1","DCA2_hist_2","DCA2_hist_3","sDCA2_hist_0","sDCA2_hist_1","sDCA2_hist_2","sDCA2_hist_3",\
                        "myvtx_hist_0","myvtx_hist_1","myvtx_hist_2",'myvtx_hist_3','myvtx_hist_4']

In [63]:
N_hists = len(hist_select_3D_names)
loc = True

In [64]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


for iFile in range(len(sim_file_names)):
    print(sim_file_path+sim_file_names[iFile])
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)

infile.Close()

input/my-m_ee_Run14AuAu_57th_new_19676_1072runs.root
../sim/output/Newembed/sngl/dalitz_50M_v8.root
../sim/output/Newembed/sngl/phi_50M_v8.root
../sim/output/Newembed/sngl/photon_100M_v2.root
../sim/output/Newembed/sngl/ccbar_50M_v2.root
../sim/output/Newembed/sngl/bbbar_10M_v0.root


In [78]:

conv_type = [1,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1250,900)
c0.Divide(2,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    dalitz_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dalitz_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(dalitz_proj,Lcolor=4)
    phi_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"phi_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(phi_proj,Lcolor=root.kGreen+2)
    conv_proj = hists_sim[2][type_loc].ProjectionX(hists_sim[2][type_loc+4].GetName()+f"conv_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2)
    ccbar_proj = hists_sim[3][type_loc].ProjectionX(hists_sim[3][type_loc+4].GetName()+f"ccbar_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=root.kMagenta)
    bbbar_proj = hists_sim[4][type_loc].ProjectionX(hists_sim[4][type_loc+4].GetName()+f"bbbar_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(bbbar_proj,Lcolor=root.kOrange+7)
    real_proj.Scale(1./(1+real_proj.Integral()))
    dalitz_proj.Scale(1./(1+dalitz_proj.Integral()))
    phi_proj.Scale(1./(1+phi_proj.Integral()))
    conv_proj.Scale(1./(1+conv_proj.Integral()))
    ccbar_proj.Scale(1./(1+ccbar_proj.Integral()))
    bbbar_proj.Scale(1./(1+bbbar_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-500,500)
    real_proj.GetYaxis().SetRangeUser(0.0001,10)
    real_proj.Draw()
    dalitz_proj.Draw("same")
    phi_proj.Draw("same")
    conv_proj.Draw("same")
    ccbar_proj.Draw("same")
    bbbar_proj.Draw("same")
    legends.append(root.TLegend(0.2,0.7,0.95,0.95))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(dalitz_proj,legend_names[1],"l")
    legends[-1].AddEntry(phi_proj,legend_names[2],"l")
    legends[-1].AddEntry(conv_proj,legend_names[3],"l")
    legends[-1].AddEntry(ccbar_proj,legend_names[4],"l")
    legends[-1].AddEntry(bbbar_proj,legend_names[5],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [90]:

conv_type = [25,25]
pt = [0.45,1.45]
c0 = root.TCanvas(f"c0",f"c0",1250,900)
c0.Divide(2,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=4)
    sum_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dalitz1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sum_proj,Lcolor=1,Msize=0, Lwidth=4)
    phi_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"phi1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(phi_proj,Lcolor=root.kGreen+2)
    conv_proj = hists_sim[2][type_loc].ProjectionX(hists_sim[2][type_loc+4].GetName()+f"conv1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2)
    ccbar_proj = hists_sim[3][type_loc].ProjectionX(hists_sim[3][type_loc+4].GetName()+f"ccbar1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=root.kMagenta)
    bbbar_proj = hists_sim[4][type_loc].ProjectionX(hists_sim[4][type_loc+4].GetName()+f"bbbar1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(bbbar_proj,Lcolor=root.kOrange+7)
    sum_proj.Add(sum_proj,-0.8)
    sum_proj.Add(phi_proj,0.00)
    sum_proj.Add(conv_proj,0.0)
    sum_proj.Add(ccbar_proj,0.04)
    sum_proj.Add(bbbar_proj,0.04)
    real_proj.Scale(1./(1+real_proj.Integral()))
    sum_proj.Scale(1./(1+sum_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-500,500)
    real_proj.GetYaxis().SetRangeUser(0.0001,10)
    real_proj.Draw("P")
    sum_proj.Draw("same H")
    legends.append(root.TLegend(0.2,0.7,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].SetNColumns(2)
    legends[-1].AddEntry(root.nullptr,"","")
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(sum_proj,"sum","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [87]:

conv_type = [25,25]
pt = [1.45,1.95]
c0 = root.TCanvas(f"c0",f"c0",1250,900)
c0.Divide(2,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real2_projproj_{icentr}_{type_loc}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=4)
    sum_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dalitz2_projproj_{icentr}_{type_loc}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sum_proj,Lcolor=1,Msize=0, Lwidth=4)
    phi_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"phi2_projproj_{icentr}_{type_loc}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(phi_proj,Lcolor=root.kGreen+2)
    conv_proj = hists_sim[2][type_loc].ProjectionX(hists_sim[2][type_loc+4].GetName()+f"conv2_proj_proj){icentr}_{type_loc}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2)
    ccbar_proj = hists_sim[3][type_loc].ProjectionX(hists_sim[3][type_loc+4].GetName()+f"ccbar2_proj_proj){icentr}_{type_loc}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=root.kMagenta)
    bbbar_proj = hists_sim[4][type_loc].ProjectionX(hists_sim[4][type_loc+4].GetName()+f"bbbar2_proj_proj){icentr}_{type_loc}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(bbbar_proj,Lcolor=root.kOrange+7)
    sum_proj.Add(sum_proj,-0.8)
    sum_proj.Add(phi_proj,0.00)
    sum_proj.Add(conv_proj,0.0)
    sum_proj.Add(ccbar_proj,0.04)
    sum_proj.Add(bbbar_proj,0.04)
    real_proj.Scale(1./(1+real_proj.Integral()))
    sum_proj.Scale(1./(1+sum_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-500,500)
    real_proj.GetYaxis().SetRangeUser(0.0001,10)
    real_proj.Draw("P")
    sum_proj.Draw("same H")
    legends.append(root.TLegend(0.2,0.7,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].SetNColumns(2)
    legends[-1].AddEntry(root.nullptr,"","")
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(sum_proj,"sum","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [ ]:

conv_type = [25,25]
pt = [1.45,1.95]
c0 = root.TCanvas(f"c0",f"c0",1250,900)
c0.Divide(2,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+4
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=4)
    sum_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dalitz1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sum_proj,Lcolor=1,Msize=0, Lwidth=4)
    phi_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"phi1_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(phi_proj,Lcolor=root.kGreen+2)
    conv_proj = hists_sim[2][type_loc].ProjectionX(hists_sim[2][type_loc+4].GetName()+f"conv1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2)
    ccbar_proj = hists_sim[3][type_loc].ProjectionX(hists_sim[3][type_loc+4].GetName()+f"ccbar1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=root.kMagenta)
    bbbar_proj = hists_sim[4][type_loc].ProjectionX(hists_sim[4][type_loc+4].GetName()+f"bbbar1_proj_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(bbbar_proj,Lcolor=root.kOrange+7)
    sum_proj.Add(sum_proj,-0.8)
    sum_proj.Add(phi_proj,0.00)
    sum_proj.Add(conv_proj,0.0)
    sum_proj.Add(ccbar_proj,0.04)
    sum_proj.Add(bbbar_proj,0.04)
    real_proj.Scale(1./(1+real_proj.Integral()))
    sum_proj.Scale(1./(1+sum_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-500,500)
    real_proj.GetYaxis().SetRangeUser(0.0001,10)
    real_proj.Draw("P")
    sum_proj.Draw("same H")
    legends.append(root.TLegend(0.2,0.7,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].SetNColumns(2)
    legends[-1].AddEntry(root.nullptr,"","")
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(sum_proj,"sum","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [67]:
conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1250,900)
c0.Divide(2,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    pos_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="DCA_{T}, #mum^{-1}",offset_y=1.1)
    neg_proj = hists_read[0][type_loc+4].ProjectionX(hists_read[0][type_loc+4].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[0][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[0][type_loc+4].ProjectionX(hists_sim[0][type_loc+4].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.GetMaximum()))
    neg_proj.Scale(1./(1+neg_proj.GetMaximum()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.GetMaximum()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.GetMaximum()))
    root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-500,500)
    pos_proj.Draw()
    neg_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [68]:

conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
legends = []
proj_y = hists_read[0][8].ProjectionY(hists_read[0][0].GetName()+f"pos_proj_y",1,-1,1,-1)
biny0 = proj_y.FindBin(-0.02)
biny1 = proj_y.FindBin(0.09)
for icentr in range(N_centr+1):
    c0.cd(1+icentr)
    type_loc = icentr+8
    pos_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"pos_proj_{icentr}",biny0,biny1,1,-1)
    Format_Hist_total(pos_proj,title_x="x, cm",offset_y=1.1)
    root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-1800,1000)
    pos_proj.Draw()
    legends.append(root.TLegend(0.5,0.6,0.95,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    print(pos_proj.Integral(pos_proj.FindBin(0.27),pos_proj.FindBin(0.4))/pos_proj.Integral())

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

0.9933806987126047
0.9938491034638606
0.9936042596679843
0.9910297724320795
0.9724796620409946


In [69]:

conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
proj_x = hists_read[0][8].ProjectionX(hists_read[0][0].GetName()+f"pos_proj_x",1,-1,1,-1)
biny0 = proj_x.FindBin(0.27)
biny1 = proj_x.FindBin(0.40)
legends = []
for icentr in range(N_centr+1):
    c0.cd(1+icentr)
    type_loc = icentr+8
    pos_proj = hists_read[0][type_loc].ProjectionY(hists_read[0][type_loc].GetName()+f"pos_proj_{icentr}",1,-1,1,-1)
    Format_Hist_total(pos_proj,title_x="y, cm",offset_y=1.1)
    root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-1800,1000)
    pos_proj.Draw()
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    print(pos_proj.Integral(pos_proj.FindBin(-0.02),pos_proj.FindBin(0.09))/pos_proj.Integral())

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

0.9997149878965969
0.9998036106026411
0.999609070072878
0.9983918881061387
0.9859360651131485


In [70]:

conv_type = [25,25]
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
legends = []
for icentr in range(N_centr+1):
    c0.cd(1+icentr)
    type_loc = icentr+8
    pos_proj = hists_read[0][type_loc].ProjectionZ(hists_read[0][type_loc].GetName()+f"pos_proj_{icentr}",1,-1,1,-1)
    Format_Hist_total(pos_proj,title_x="y, cm",offset_y=1.1)
    root.gPad.SetLogy()
    pos_proj.GetXaxis().SetRangeUser(-1800,1000)
    pos_proj.Draw()
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")